## Exercises

- create a list of 3x3 matrices with random integer coefficients
- what is the probability that such a matrix is invertible if the coefficients are 0 and 1 ?
- use ```np.linalg.eigvals``` to find the biggest eigenvalue for the matrices

---

- make a [van der monde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix)
with first row a random list of 5 real numbers
- calculate its determinant and verify the formula



In [4]:
import numpy as np
from numpy.random import randint

#?randint

```Docstring:
randint(low, high=None, size=None, dtype=int)

Return random integers from `low` (inclusive) to `high` (exclusive).

Return random integers from the "discrete uniform" distribution of
the specified dtype in the "half-open" interval [`low`, `high`). If
`high` is None (the default), then results are from [0, `low`).
```


In [7]:
randint(0,2,9)

array([1, 1, 1, 1, 0, 1, 1, 1, 0])

In [8]:
randint(0,2,9).reshape(3,3)

array([[0, 0, 1],
       [1, 0, 0],
       [1, 1, 1]])

In [9]:
from numpy.linalg import det

# Calculating the probability by simulation

this is easiest as it's just one line of code

we can calculate it exactly as there are only $2^9 = 512$
matrices 

In [12]:
[det(randint(0,2,9).reshape(3,3)) for k in range(10)]

[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0]

# This works

but you should **never** test for a value of a floating point number

``` x == 0.0 ``` is **bad**

you should test for being in an interval

``` abs(x) < 0.1 ``` is **good**

In [13]:
[det(randint(0,2,9).reshape(3,3)) for k in range(10)].count(0.0)

6

In [18]:
num_tries =  10**5
X = [ 1 for k in range(num_tries) if abs(det(randint(0,2,9).reshape(3,3))) < 0.5 ]
len(X)/num_tries

0.65956

# Calculating the probability exactly

we can do this by enumerating all the matrices

- this can be done using [recursion](https://realpython.com/python-thinking-recursively/#:~:text=Recursive%20Functions%20in%20Python,-Now%20that%20we&text=A%20recursive%20function%20is%20a,met%20to%20return%20a%20result.)
- but I'll do it using loops



---

this was my first attempt at making a list of all the possibilities

it doesn't work because ```L``` is a list of lists 
so when I try to copy the elements using ```L[:]```
- I don't get a **real** copy of the lists 
- I just get a copy of **references** to the lists

[shallow vs deep copy](https://betterprogramming.pub/shallow-copy-vs-deep-copy-in-python-357e5f502bf9?gi=a13c5af07d2a#:~:text=In%20Python%2C%20a%20shallow%20copy,copies%20of%20the%20child%20objects.)

In [32]:
L = [[0],[1]]

for k in range(1):
    M = []
    for dd in [0,1]:
        tmp = L[:]
        print('>',L)
        for x in tmp:
            x.append(dd)
        print(tmp)
        M.extend(tmp)
        print(M)
    L = M
        
    

> [[0], [1]]
[[0, 0], [1, 0]]
[[0, 0], [1, 0]]
> [[0, 0], [1, 0]]
[[0, 0, 1], [1, 0, 1]]
[[0, 0, 1], [1, 0, 1], [0, 0, 1], [1, 0, 1]]


In [33]:
import copy

In [35]:
L = [[0],[1]]

for k in range(1):
    M = []
    for dd in [0,1]:
        tmp = copy.deepcopy(L)
        print('>',L)
        for x in tmp:
            x.append(dd)
        print(tmp)
        M.extend(tmp)
        print(M)
    L = M
        

> [[0], [1]]
[[0, 0], [1, 0]]
[[0, 0], [1, 0]]
> [[0], [1]]
[[0, 1], [1, 1]]
[[0, 0], [1, 0], [0, 1], [1, 1]]


In [50]:
L = [[0],[1]]

for k in range(8):
    M = []
    # add either 0 or 1 to 
    for dd in [0,1]:
        tmp = copy.deepcopy(L)
        for x in tmp:
            x.append(dd)
        M.extend(tmp)
        
    L = M
        

```[0, 0, 1, 1, 1, 1, 1, 1, 1]```

should be the represenation of 127 in binary 

I'll use this to give another solution below

In [54]:
len(L), [ x for x in reversed(L[127]) ]

(512, [0, 0, 1, 1, 1, 1, 1, 1, 1])

In [56]:
DD = [ det(np.array(x).reshape(3,3)) for x in L]

In [59]:
len([1 for x in DD if abs(x) < .2])/512

0.66015625

# Using binary expansions

There is an easy algorithm that you should know
for writing a number $x$ in base $b$

## Exo 

Modify my function ```f``` to write $x$ in base $5$



In [64]:
def f(x, N=3):
    ```calculate the fixed width binary expansion of x```
    L = []
    for k in range(N):
        L.append(x % 2)
        x = x // 2
    return L

N = 9
L = [ f(x,N=N) for x in range(2**N)]


In [65]:
DD = [ det(np.array(x).reshape(3,3)) for x in L]
len([1 for x in DD if abs(x) < .2])/512

0.66015625